In [1]:
from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.simplefilter("ignore", ResourceWarning)

# SearchFlow Demo 👨🏻‍💻
This notebook demonstrates the functions for importing data from various sources. 
Loading it into a VectorStore, and then using it to answer questions with a Retrieval Augemented Reasoning  🦜🔗 LangGraph.

## Creating a new project



In [5]:
from searchflow.db import DB
db = DB()

print(db.list_projects())
db.create_project("Test", description="This is a test project")

[]


'Test'

In [4]:
db.remove_project("Test")

True

## Importing Data
### 1. From a URL 🔗

**Web Crawling and Data Extraction Example**


In [ ]:
from searchflow.importers import WebScraper

scraper = WebScraper(project_name='Test', db=db)
scraper.get_all_links("https://vectrix.ai")

In [ ]:
from searchflow.importers import WebScraper

scraper = WebScraper(project_name='Test', db=db)
scraper.full_import("https://dataframe.be", max_pages=100)

In [ ]:
print(db.get_links_to_confirm("Test"))

In [ ]:
db.add_links_to_index(links=["https://langchain-ai.github.io/langgraph/reference/graphs/"],base_url="https://langchain-ai.github.io/langgraph/reference/graphs/", project_name="Test", status="Confirm page import")

In [ ]:
print(db.get_indexing_status("Test"))

In [ ]:
from searchflow.importers import WebScraper

scraper = WebScraper(project_name='Test', db=db)

confirmed_links = [link['url'] for link in db.get_links_to_confirm("Test")]
scraper.download_pages(confirmed_links, project_name="Test")

### 2. Upload files ⬆️
You can also upload files and add them to the vector store, Vectrix will automaticly detect the file type extract the text and chunk the content into blocks.

In [8]:
from searchflow.importers import Files
file = './files/pdf_with_scannedtext.pdf'

# Load bytes data
with open(file, "rb") as f:
    bytes_data = f.read()

files = Files()
files.upload_file(
    document_data=[(bytes_data, "test.pdf")],
    project_name="Test",
    inference_type="local"
)

2024-09-03 17:52:34,360 - Files - INFO - Processing files locally
2024-09-03 17:52:34,909 - Files - INFO - Processing file 1 of 1
2024-09-03 17:52:34,960 - Files - INFO - Uploaded test.pdf to object storage
2024-09-03 17:53:00,509 - Files - INFO - Chunked the document into 1 parts


In [ ]:
# Remove a file
db.remove_file("Test", "test.pdf")

### 3. Chrome Plugin 🦊

In [ ]:
# Launch the FastAPI server
#!python src/searchflow/api.py



In [164]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from searchflow.db import DB
from langchain import hub
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
os.environ["LANGCHAIN_TRACING_V2"] = "true"



llm = llm.bind_tools([QueryResult], tool_choice="required", strict=True)

@tool
def run_sql_query(query: str) -> str:
    """
    Run a SQL query on the database.
    """
    db = DB()
    result = db.run_query(query)
    return result


prompt = hub.pull("write_query")

chain = prompt | llm 

response = chain.invoke({"USER_QUESTION": "How many webpages are there?"}).tool_calls
print(response[0]['args']['query'])

SELECT COUNT(*) AS total_webpages FROM document_metadata WHERE file_type = 'webpage';


In [165]:
answer = run_sql_query(response[0]['args']['query'])
print(answer.fetchall())

[(47,)]


## SQL Database Chain


In [174]:
from langchain_openai import OpenAI
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from searchflow.db import DB

In [188]:
db = DB()
db = SQLDatabase(db.engine)
llm = OpenAI()

In [189]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True)

In [192]:
db_chain.run("How many blog posts are writteb by Dimitri Allaert ?")



> Entering new SQLDatabaseChain chain...
How many blog posts are writteb by Dimitri Allaert ?
SQLQuery:SELECT COUNT(*) 
FROM document_metadata 
WHERE author = 'Dimitri Allaert' AND file_type = 'blog_post';
SQLResult: [(0,)]
Answer:0
> Finished chain.


'0'

In [214]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following table:
DDL:
CREATE TABLE document_metadata (
    id SERIAL PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    author VARCHAR(255),
    file_type VARCHAR(50),
    word_count INTEGER,
    language VARCHAR(50),
    source VARCHAR(255),
    content_type VARCHAR(100),
    tags TEXT[],  -- Array of strings
    summary TEXT,
    url VARCHAR(255),
    project_name VARCHAR(255),
    indexing_status VARCHAR(50),
    filename VARCHAR(255),
    priority INTEGER,
    read_time FLOAT,  -- In minutes
    creation_date TIMESTAMPTZ,
    last_modified_date TIMESTAMPTZ,
    upload_date TIMESTAMPTZ DEFAULT CURRENT_TIMESTAMP,
    CONSTRAINT uq_doc_url_project UNIQUE (url, project_name),
    CONSTRAINT fk_project_name FOREIGN KEY (project_name) REFERENCES projects(name)
);

Content:
id,title,author,file_type,word_count,language,source,content_type,tags,summary,url,project_name,indexing_status,filename,priority,read_time,creation_date,last_modified_date,upload_date
8,AI Expertisecentrum,,webpage,408,NL,chrome_extension,blog_post,"[""AI"",""Vlaamse overheid"",""digitale transformatie"",""ethiek"",""kennisdeling""]",Het AI Expertisecentrum ondersteunt de Vlaamse overheid en lokale overheden in het gebruik van artificiële intelligentie om de efficiëntie en innovatie van diensten te verbeteren. Het centrum biedt een kader voor AI-toepassingen en stimuleert kennisdeling binnen de overheid.,https://www.vlaanderen.be/digitaal-vlaanderen/onze-oplossingen/ai-expertisecentrum,Test,,AI Expertisecentrum,,2.04,,,2024-09-03 16:27:05.904255+00
9,Wat betekent de aankomende NIS2-richtlijn (cybersecurity wetgeving)…,,webpage,889,NL,chrome_extension,blog_post,"[""Cybersecurity"",""NIS2"",""GDPR"",""Belgium"",""Government"",""Digital Security""]","The NIS2 directive, effective from October 2024 in Belgium, aims to enhance cybersecurity across Europe, impacting government leaders by imposing strict security standards similar to GDPR. It emphasizes the responsibility of organizations in managing digital security, requiring comprehensive risk management, incident response plans, and a culture of security awareness among employees.",https://www.vlaanderen.be/digitaal-vlaanderen/wat-betekent-de-aankomende-nis2-richtlijn-cybersecurity-wetgeving-voor-leidinggevenden-bij-de-overheid,Vlaamse Overheid,,Wat betekent de aankomende NIS2-richtlijn (cybersecurity wetgeving)…,,4.445,,,2024-09-03 16:30:03.524486+00
10,Vectrix Mail,,webpage,5070,EN,chrome_extension,email,"[""AI"",""Community"",""Event"",""Newsletter"",""Invitation""]","The content consists of various email communications regarding community events, invitations, confirmations, and newsletters related to AI and business activities. Key participants include Paulien Derden and Dimitri Allaert, with discussions about supporting an AI community in Antwerp, event invitations, and confirmations for conferences and meetings.",https://mail.google.com/mail/u/0/#search/bart/FMfcgzQVzNvVshgTnGNKFwPdtwZrsXNt,Vlaamse Overheid,,Vectrix Mail,,25.35,,,2024-09-03 16:32:39.998075+00
11,Vectrix - SLM Training & AI Solutions,,webpage,126,EN,webpage,other,"[""AI"",""Technology"",""Business"",""Product Development""]","The content discusses an AI solution development process, including initial discussions, feasibility checks, MVP development, and final product creation, emphasizing security, compliance, and user control.",https://vectrix.ai/,Vectrix,,,,0.63,,,2024-09-03 17:00:00.472415+00


If someone asks for the table foobar, they really mean the employee table.

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "dialect"], template=_DEFAULT_TEMPLATE
)

db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True)

In [228]:

print(db_chain.run("What are all the various sources of content in the data ?"))



> Entering new SQLDatabaseChain chain...
What are all the various sources of content in the data ?
SQLQuery:SELECT DISTINCT source FROM document_metadata;
SQLResult: [('chrome_extension',), ('webpage',), ('uploaded_file',)]
Answer:The various sources of content in the data are: chrome_extension, webpage, and uploaded_file.
> Finished chain.
The various sources of content in the data are: chrome_extension, webpage, and uploaded_file.
